In [7]:
from ucimlrepo import fetch_ucirepo

import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [8]:
# fetch dataset 
adult = fetch_ucirepo(id=2) 

# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

df = X.join(y)

# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables)

# drop rows with missing values
df = df.dropna()

# drop duplicates
df = df.drop_duplicates()

# drop redundant columns
df = df.drop('education', axis=1)

# Format column, get rid of variations
df['income'] = df['income'].replace('>50K.', '>50K')
df['income'] = df['income'].replace('<=50K.', '<=50K')

# Binarize column
df['income'] = np.where(df['income'] == '>50K', 1, 0)
df.rename(columns={'income':'income>50K'}, inplace=True)

# Extract features and target
df3 = df.copy()
y3 = df3.iloc[:,-1:]
X3 = df3.iloc[:,1:-1]

# Create encoder
encoder = LabelEncoder()

# Encode categorical data
X3['sex'] = encoder.fit_transform(X3['sex'])

# One hot encode categorical data - marital-status
X3['marital-status'].replace(['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse'], 'Married', inplace=True)
X3['marital-status'].replace(['Divorced', 'Separated', 'Widowed'], 'Divorced', inplace=True)
X3 = pd.get_dummies(X3, columns=['marital-status'], dtype=int)

# One hot encode categorical data - workclass
X3['workclass'].replace(['Self-emp-not-inc', 'Self-emp-inc'], 'self-emp', inplace=True)
X3['workclass'].replace(['Federal-gov', 'Local-gov', 'State-gov'], 'government', inplace=True)
X3['workclass'].replace(['Never-worked', 'Without-pay'], 'jobless', inplace=True)
X3 = pd.get_dummies(X3, columns=['workclass'], dtype=int)

# One hot encode categorical data - occupation
X3['occupation'].replace(['Tech-support', 'Craft-repair', 'Machine-op-inspct'], 'Technical/Support', inplace=True)
X3['occupation'].replace(['Other-service', 'Priv-house-serv', 'Protective-serv'], 'Service', inplace=True)
X3['occupation'].replace(['Exec-managerial', 'Adm-clerical'], 'Management/Administration', inplace=True)
X3['occupation'].replace(['Sales'], 'Sales', inplace=True)
X3['occupation'].replace(['Prof-specialty'], 'Professional', inplace=True)
X3['occupation'].replace(['Handlers-cleaners', 'Farming-fishing', 'Transport-moving'], 'Manual Labor', inplace=True)
X3['occupation'].replace(['Armed-Forces'], 'Specialized', inplace=True)
X3 = pd.get_dummies(X3, columns=['occupation'], dtype=int)

# One hot encode categorical data - relationship
X3['relationship'].replace(['Wife', 'Husband'], 'Spouse', inplace=True)
X3['relationship'].replace(['Own-child'], 'Child', inplace=True)
X3['relationship'].replace(['Other-relative'], 'Other Relatives', inplace=True)
X3['relationship'].replace(['Not-in-family', 'Unmarried'], 'Non-Family', inplace=True)
X3 = pd.get_dummies(X3, columns=['relationship'], dtype=int)

# One hot encode categorical data - relationship
X3['race'].replace(['White'], 'White', inplace=True)
X3['race'].replace(['Asian-Pac-Islander'], 'Asian/Pacific Islander', inplace=True)
X3['race'].replace(['Amer-Indian-Eskimo'], 'American Indian/Eskimo', inplace=True)
X3['race'].replace(['Black'], 'Black', inplace=True)
X3['race'].replace(['Other'], 'Other', inplace=True)
X3 = pd.get_dummies(X3, columns=['race'], dtype=int)

X3['native-country'].replace(['United-States', 'Canada', 'Outlying-US(Guam-USVI-etc)', 'Puerto-Rico', 'Mexico', 'Cuba', 'Jamaica', 'Dominican-Republic', 'Haiti', 'Guatemala', 'Honduras', 'El-Salvador', 'Nicaragua', 'Trinadad&Tobago', 'Panama'], 'North America', inplace=True)
X3['native-country'].replace(['England', 'Germany', 'Greece', 'Italy', 'Poland', 'Portugal', 'Ireland', 'France', 'Scotland', 'Yugoslavia', 'Hungary', 'Holand-Netherlands'], 'Europe', inplace=True)
X3['native-country'].replace(['Cambodia', 'India', 'Japan', 'China', 'Philippines', 'Vietnam', 'Taiwan', 'Laos', 'Iran', 'Thailand', 'Hong'], 'Asia', inplace=True)
X3['native-country'].replace(['Ecuador', 'Columbia', 'Peru'], 'Other', inplace=True)

X3 = pd.get_dummies(X3, columns=['native-country'], dtype=int)

display(X3.iloc[:4, :])

{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Mon Aug 07 2023', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the following conditions: ((AAG

,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,marital-status_Divorced,marital-status_Married,marital-status_Never-married,workclass_?,...,race_Asian/Pacific Islander,race_Black,race_Other,race_White,native-country_?,native-country_Asia,native-country_Europe,native-country_North America,native-country_Other,native-country_South
0,77516,13,1,2174,0,40,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1,83311,13,1,0,0,13,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
2,215646,9,1,0,0,40,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,234721,7,1,0,0,40,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define a model-building function
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_train_scaled.shape[1],)))
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='keras_tuner_demo'
)

# Perform the hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=1)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy}")



Epoch 1/100


952/952 [==============================] - 3s 2ms/step - loss: 0.3528 - accuracy: 0.8338 - val_loss: 0.3385 - val_accuracy: 0.8402
Epoch 2/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3280 - accuracy: 0.8475 - val_loss: 0.3350 - val_accuracy: 0.8408
Epoch 3/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3252 - accuracy: 0.8491 - val_loss: 0.3328 - val_accuracy: 0.8436
Epoch 4/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3224 - accuracy: 0.8517 - val_loss: 0.3339 - val_accuracy: 0.8419
Epoch 5/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3207 - accuracy: 0.8520 - val_loss: 0.3357 - val_accuracy: 0.8432
Epoch 6/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3193 - accuracy: 0.8511 - val_loss: 0.3336 - val_accuracy: 0.8424
Epoch 7/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3177 - accuracy: 0.8517 - val_loss: 0.3335 - val_accuracy: 0.

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=42)

# Feature scaling for neural networks
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Common parameters
input_shape = (X_train_scaled.shape[1],)  # Input shape
n_classes = 1  # Change accordingly if you have a different problem
activation_hidden = 'relu'  # Activation function for hidden layers
activation_output = 'sigmoid'  # Activation function for the output layer (binary classification)
loss_function = 'binary_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']

In [11]:
model_1_layer = Sequential([
    Dense(64, activation=activation_hidden, input_shape=input_shape),
    Dense(n_classes, activation=activation_output)
])

model_1_layer.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Train the model
history_1_layer = model_1_layer.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)



Epoch 1/100


952/952 [==============================] - 3s 2ms/step - loss: 0.3623 - accuracy: 0.8277 - val_loss: 0.3371 - val_accuracy: 0.8423
Epoch 2/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3287 - accuracy: 0.8476 - val_loss: 0.3311 - val_accuracy: 0.8449
Epoch 3/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3252 - accuracy: 0.8490 - val_loss: 0.3307 - val_accuracy: 0.8454
Epoch 4/100
952/952 [==============================] - 2s 3ms/step - loss: 0.3238 - accuracy: 0.8492 - val_loss: 0.3315 - val_accuracy: 0.8431
Epoch 5/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3221 - accuracy: 0.8507 - val_loss: 0.3311 - val_accuracy: 0.8446
Epoch 6/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3210 - accuracy: 0.8506 - val_loss: 0.3333 - val_accuracy: 0.8414
Epoch 7/100
952/952 [==============================] - 8s 8ms/step - loss: 0.3206 - accuracy: 0.8509 - val_loss: 0.3315 - val_accuracy: 0.

In [12]:
model_3_layers = Sequential([
    Dense(64, activation=activation_hidden, input_shape=input_shape),
    Dense(64, activation=activation_hidden),
    Dense(64, activation=activation_hidden),
    Dense(n_classes, activation=activation_output)
])

model_3_layers.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Train the model
history_3_layers = model_3_layers.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/100
952/952 [==============================] - 4s 3ms/step - loss: 0.3510 - accuracy: 0.8364 - val_loss: 0.3402 - val_accuracy: 0.8390
Epoch 2/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3285 - accuracy: 0.8468 - val_loss: 0.3420 - val_accuracy: 0.8390
Epoch 3/100
952/952 [==============================] - 2s 3ms/step - loss: 0.3252 - accuracy: 0.8481 - val_loss: 0.3333 - val_accuracy: 0.8420
Epoch 4/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3228 - accuracy: 0.8497 - val_loss: 0.3371 - val_accuracy: 0.8431
Epoch 5/100
952/952 [==============================] - 2s 3ms/step - loss: 0.3203 - accuracy: 0.8525 - val_loss: 0.3332 - val_accuracy: 0.8461
Epoch 6/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3184 - accuracy: 0.8513 - val_loss: 0.3370 - val_accuracy: 0.8454
Epoch 7/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3164 - accuracy: 0.8525 - val_loss: 0.3334 - val_accuracy: 0.8435

In [13]:
model_7_layers = Sequential([
    Dense(64, activation=activation_hidden, input_shape=input_shape),
    Dense(64, activation=activation_hidden),
    Dense(64, activation=activation_hidden),
    Dense(64, activation=activation_hidden),
    Dense(64, activation=activation_hidden),
    Dense(64, activation=activation_hidden),
    Dense(64, activation=activation_hidden),
    Dense(n_classes, activation=activation_output)
])

model_7_layers.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Train the model
history_7_layers = model_7_layers.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/100
952/952 [==============================] - 5s 3ms/step - loss: 0.3519 - accuracy: 0.8360 - val_loss: 0.3346 - val_accuracy: 0.8420
Epoch 2/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3294 - accuracy: 0.8469 - val_loss: 0.3374 - val_accuracy: 0.8411
Epoch 3/100
952/952 [==============================] - 3s 3ms/step - loss: 0.3255 - accuracy: 0.8486 - val_loss: 0.3398 - val_accuracy: 0.8403
Epoch 4/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3237 - accuracy: 0.8511 - val_loss: 0.3327 - val_accuracy: 0.8439
Epoch 5/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3210 - accuracy: 0.8505 - val_loss: 0.3348 - val_accuracy: 0.8440
Epoch 6/100
952/952 [==============================] - 3s 3ms/step - loss: 0.3196 - accuracy: 0.8525 - val_loss: 0.3352 - val_accuracy: 0.8433
Epoch 7/100
952/952 [==============================] - 2s 2ms/step - loss: 0.3181 - accuracy: 0.8524 - val_loss: 0.3365 - val_accuracy: 0.8439

In [14]:
# Evaluate the models
test_loss, test_accuracy = model_1_layer.evaluate(X_test_scaled, y_test, verbose=1)
print(f"1 Layer Model - Test Accuracy: {test_accuracy}")

test_loss, test_accuracy = model_3_layers.evaluate(X_test_scaled, y_test, verbose=1)
print(f"3 Layers Model - Test Accuracy: {test_accuracy}")

test_loss, test_accuracy = model_7_layers.evaluate(X_test_scaled, y_test, verbose=1)
print(f"7 Layers Model - Test Accuracy: {test_accuracy}")

298/298 [==============================] - 0s 1ms/step - loss: 0.3426 - accuracy: 0.8453
1 Layer Model - Test Accuracy: 0.8452568650245667
298/298 [==============================] - 0s 1ms/step - loss: 0.7124 - accuracy: 0.8341
3 Layers Model - Test Accuracy: 0.8341212272644043
298/298 [==============================] - 0s 1ms/step - loss: 0.6283 - accuracy: 0.8308
7 Layers Model - Test Accuracy: 0.8307595252990723
